In this mini-course, we continue to study bootstrap. We mainly focus on constructing bootstrap confidence intervals and hypothesis testing. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random

from scipy import stats
from scipy.stats import norm
from sklearn.utils import resample
#from scipy.stats import iqr

%matplotlib inline

### I. The Smooth Function Model 

Bhattacharya andn Ghosh (1978) intorduced the **smooth function model** and Hall (1992) generalized it. Here, we will discuss a simple version of it. And show that we can present majority of the common test statistic in this manner. This will help us generate the bootstrap confidence interval.

The model looks like the following. Let $w=(w_{1}, w_{2},...w_{n})'$ be an i.i.d. random sample from a distribution $F$, where the CDF function itself is unknown and each $w_{i}$ is a d-dimensional random vector in nature. Let $\hat{\theta}_{n}$ be an estimator of the parameter of interest $\theta$, and define $(\mu, \sigma^{2})$  to be the mean and asymptotic variance of the estimator $\hat{\theta}_{n}$ and let $(\hat{\mu}, \hat{\sigma}_{n}^{2})$ be their respective estimates with 
$\mu=E(w_{i}) \in \mathbb{R}^{d}$ and $\sigma^{2}=lim_{n \rightarrow \infty}Var(\sqrt{n}\hat{\theta}_{n}) \in \mathbb{R}$. Let both $g(.), h(.)$ be real-valued smooth functions whose domain is $\mathbb{R}^{d}$. Define $\theta=g(\mu)$, $\hat{\theta}_{n}=g(\hat{\mu})$, $\sigma=h(\mu)>0$ and $h(\hat{\mu})=\hat{\sigma}_{n}$. 

It turns out many times we can apply this framework during inference. Below are some examples. The strategy is that if we are given $\theta$, we need to find $w_{i},g(.),h(.)$:

##### Example:
Suppose $x=(x_{1}, x_{2},...x_{n})'$ is an i.i.d. random sample from a distribution $F$, with $\theta=E(x_{i})$. We are interested in estimating $\theta$. To fit into the smooth model, we can define $w_{i}=(x_{i}, x^{2}_{i})' \in \mathbb{R}^{2}$, and we define $\mu=E(w_{i})=(E(x_{i}), E(x^{2}_{i}))'=(\theta, \theta^{2}+Var(x_{i}))'=(\theta, \theta^{2}+\tau^{2})'$ with $\tau^{2}=Var(x_{i})$. Following the definitions above, we can write down the sample analog such that 
$\hat{\mu}=\frac{1}{n}\sum_{i=1}^{n}w_{i}$, $\hat{\theta}_{n}=\frac{1}{n}\sum_{i=1}^{n}x_{i}$, and $\hat{\sigma}_{n}=\sqrt{\frac{1}{n}\sum_{i=1}^{n}x^{2}_{i}-(\frac{1}{n}\sum_{i=1}^{n}x_{i})^{2}}$. Now define $g(x_{1},x_{2})=x_{1}$, $h(x_{1},x_{2})=\sqrt{x_{2}-x^{2}_{1}}$. One can verify that $g(\mu)=\theta$ and $g(\hat{\mu})=g(\frac{1}{n}\sum_{i=1}^{n}w_{i})=\frac{1}{n}\sum_{i=1}^{n}x_{i}=\hat{\theta}_{n}$. Similarly, we know that $Var(\sqrt{n}\frac{1}{n}\sum_{i=1}^{n}x_{i}) =\tau^{2}$. So $h(x_{1},x_{2})=\sqrt{x_{2}-x^{2}_{1}}$ implies that $h(\mu)=\sqrt{\theta^{2}+\tau^{2}-(\theta)^{2}}=\tau=\sqrt{Var(\sqrt{n}\frac{1}{n}\sum_{i=1}^{n}x_{i})}=\sqrt{Var(\sqrt{n}\hat{\theta}_{n})}=\sqrt{lim_{n \rightarrow \infty}Var(\sqrt{n}\hat{\theta}_{n}})=\sigma$ (because sending to infinity doens't matter as it gets canceld out). Meanwhile, $h(\hat{\mu})=\sqrt{\frac{1}{n}\sum_{i=1}^{n}x^{2}_{i}-(\frac{1}{n}\sum_{i=1}^{n}x_{i})^{2}}=\hat{\sigma}_{n}$. Clearly, both $g(.)$ and $h(.)$ are smooth. This confirms our smooth model.

The above is simply an example. In fact, many common statistic such as mean, variance, correlation, skewness, kurtosis, rates of means, ratio of variances can all be fit into this smooth model. However, note that quantiles and modes are not in the smooth function model.

##### Definition:
Let $y$ be a random variable with CDF $F$. For any $\alpha \in (0,1]$, the $\alpha$th quantile of a distribution $F(t)$ is defined to be $q_{\alpha}=inf\{t:F(t) \ge \alpha\}$. When $F(.)$ is strictly increasing, we have $F(q_{\alpha})=Pr(y \le q_{\alpha})=\alpha$ and hence $q_{\alpha}=F^{-1}(\alpha)$. (c.f. Bruce Hansen's manuscript, chapter 10 "Resampling Method")

We now turn to confidence intervals, and we try to incorporate our analysis into this smooth model framework. Recall that CLT states the following results (assuming $\Phi(.)$ is the CDF of the standard normal distribution and $\Phi'(t)=\phi(t)$):

   1. If $G_{n}(t)=Pr(\frac{\sqrt{n}(\hat{\theta}_{n}-\theta)}{\sigma} \le t)$, then by CLT, $lim_{n \rightarrow \infty}G_{n}(t)=\Phi(t)$.
   2. If $H_{n}(t)=Pr(\frac{\sqrt{n}(\hat{\theta}_{n}-\theta)}{\hat{\sigma}_{n}} \le t)$, then by CLT, $lim_{n \rightarrow \infty}H_{n}(t)=\Phi(t)$.
   
The confidence intervals based on the two above results can be derived easily. If we define $q_{\alpha}$ to be the $\alpha$th quantile of $G_{n}(t)$ or $H_{n}(t)$, then our $100\alpha\%$ confidence interval for $\theta$ are given by:

   1. $(\hat{\theta}_{n}-\frac{\sigma}{\sqrt{n}}q_{\frac{1+\alpha}{2}}, \hat{\theta}_{n}-\frac{\sigma}{\sqrt{n}}q_{\frac{1-\alpha}{2}})$
   2. $(\hat{\theta}_{n}-\frac{\hat{\sigma}_{n}}{\sqrt{n}}q_{\frac{1+\alpha}{2}}, \hat{\theta}_{n}-\frac{\hat{\sigma}_{n}}{\sqrt{n}}q_{\frac{1-\alpha}{2}})$
   
The bootstrap confidence intervals are inspired by these ordinary confidence intervals. 

### II. Efron's Bootstrap Percentile Interval

Efron originally treated the sampoling distribution as a posterior distribution by computing the credible interval. 

##### Definition:
Let $J_{n}(t)=Pr(\hat{\theta}_{n} \le t|x \sim F)$ and $\hat{J}_{n}(t)=Pr^{*}(\hat{\theta}_{n}^{*} \le t|x^{*} \sim \bar{F}_{n})$. Then the approximate $100\alpha\%$ confidence interval for the population parameter $\theta$ (**Efron's percentile confidence interval for the population parameter**) is given by: $(\hat{J}_{n}^{-1}(\frac{1-\alpha}{2}), \hat{J}_{n}^{-1}(\frac{1+\alpha}{2}))$. To implement this using bootstrap, we can just replace the confidence interval cutoff points with the empirical corresponding percentile cutoff points. This confidence interval is the quick-and-dirty solution using bootstrap. It doesn't involve calculating any standard errors. 

Davison and Hinkley (1997) used this concept to further develop the **reverse bootstrap percentile interval for the population parameter**, also called the **centered bootstrap confidence interval**. It is based on the distribution of $\hat{\theta}_{n}-\theta$. Essentially we use the fact that $Pr(\hat{\theta}_{n}-\theta)$ can be approximated by $Pr(\hat{\theta}_{n}^{*}-\hat{\theta}_{n})$. And thus $\theta \in (2\hat{\theta}_{n}-q_{\frac{1+\alpha}{2}}, 2\hat{\theta}_{n}-q_{\frac{1-\alpha}{2}})$ denotes the $100\alpha$% confidence interval where $q_{\alpha}$ denotes the quantile of the bootstrap distribution $\hat{\theta}_{n}^{*}$ (notice this is NOT the quantile of $\hat{\theta}_{n}^{*}-\hat{\theta}_{n}$ therefore the coefficient 2). This is discussed in Davidson and Hinkley (1997). See also https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4784504/#:~:text=Reverse%20Bootstrap%20Percentile%20Interval,like%20those%20shown%20just%20below.

##### Proposition:
Suppose there exists a monotonically increasing transformation $\psi(.)$ s.t. $Pr(\psi(\hat{\theta}_{n})-\psi({\theta}) \le t)=\Psi(t)$, where $\Psi(.)$ satisfies the following:
   1. $\Psi(t)$ is continuous and monotonically increasing. 
   2. $\Psi(t)$ is symmetric, so that $\Psi(t)=1-\Psi(-t)$.

If we know the structure of $\psi(.)$, then the approximate $100\alpha\%$ confidence interval for $\theta$ is given by the interval $(\psi^{-1}(\psi(\hat{\theta}_{n})+\Psi_{\frac{1-\alpha}{2}}), \psi^{-1}(\psi(\hat{\theta}_{n})+\Psi_{\frac{1+\alpha}{2}}))$, where $\Psi_{\alpha}$ denotes the $\alpha$th quantile of $\Psi(.)$, and remember that $\Psi(\Psi_{\alpha})=\Psi(\Psi^{-1}(\alpha))=\alpha$.

The proof of this is fairly easy. We will omit here. The $\psi(.)$ function is sort of like a 'variance stabilizing transformation' here. Now tying this results back to the smooth function model, recall that the following are true:

   1. If $G_{n}(t)=Pr(\frac{\sqrt{n}(\hat{\theta}_{n}-\theta)}{\sigma} \le t)$, then by CLT, $lim_{n \rightarrow \infty}G_{n}(t)=\Phi(t)$. This means the confidence interval for $\theta$ is given by $(\hat{\theta}_{n}-\frac{\sigma}{\sqrt{n}}q_{\frac{1+\alpha}{2}}, \hat{\theta}_{n}-\frac{\sigma}{\sqrt{n}}q_{\frac{1-\alpha}{2}})$ when $\sigma$ is known. 
   2. If $H_{n}(t)=Pr(\frac{\sqrt{n}(\hat{\theta}_{n}-\theta)}{\hat{\sigma}_{n}} \le t)$, then by CLT, $lim_{n \rightarrow \infty}H_{n}(t)=\Phi(t)$. This means the confidence interval for $\theta$ is given by $(\hat{\theta}_{n}-\frac{\hat{\sigma}_{n}}{\sqrt{n}}q_{\frac{1+\alpha}{2}}, \hat{\theta}_{n}-\frac{\hat{\sigma}_{n}}{\sqrt{n}}q_{\frac{1-\alpha}{2}})$ when $\sigma$ is unknown.
   
Now if you look at these two confidence intervals and compare them with the percentile's method confidence interval, you will see a problem. Suppose we let $t_{\alpha}$ denote the percentile such that $Pr(\hat{\theta}_{n} \le t_{\alpha})=\alpha$. Then the percentile method is given by $(t_{\frac{1-\alpha}{2}}, t_{\frac{1+\alpha}{2}})$. You can now get more specific about the form of $t_{\alpha}$. Let $g_{\alpha}$ be the $\alpha$th percentile of $G_{n}(.)$. So $Pr(\hat{\theta}_{n} \le t_{\alpha})=Pr(\frac{\sqrt{n}(\hat{\theta}_{n}-\theta)}{\sigma} \le \frac{\sqrt{n}(t_{\alpha}-\theta)}{\sigma})=G_{n}(\frac{\sqrt{n}(t_{\alpha}-\theta)}{\sigma})$. This means that $g_{\alpha}=\frac{\sqrt{n}(t_{\alpha}-\theta)}{\sigma}$ and $t_{\alpha}=\theta+\frac{\sigma}{\sqrt{n}}g_{\alpha}$. So replacing these into the original confidence interval, we have $(t_{\frac{1-\alpha}{2}}, t_{\frac{1+\alpha}{2}})=(\theta+\frac{\sigma}{\sqrt{n}}g_{\frac{1-\alpha}{2}}, \theta+\frac{\sigma}{\sqrt{n}}g_{\frac{1+\alpha}{2}})$. So a natural estimate of this confidence interval for $(t_{\frac{1-\alpha}{2}}, t_{\frac{1+\alpha}{2}})$ is $(\hat{t}_{\frac{1-\alpha}{2}}, \hat{t}_{\frac{1+\alpha}{2}})=(\hat{\theta}_{n}+\frac{\hat{\sigma}_{n}}{\sqrt{n}}g_{\frac{1-\alpha}{2}}, \hat{\theta}_{n}+\frac{\hat{\sigma}_{n}}{\sqrt{n}}g_{\frac{1+\alpha}{2}})$. Now notice the difference here between this interval and the one deduced from 2) above related to the distribution of $H_{n}(.)$:
   * $(\hat{\theta}_{n}+\frac{\hat{\sigma}_{n}}{\sqrt{n}}q_{\frac{1-\alpha}{2}}, \hat{\theta}_{n}+\frac{\hat{\sigma}_{n}}{\sqrt{n}}q_{\frac{1+\alpha}{2}})$ (the correct interval based on CLT)
   * $(\hat{\theta}_{n}-\frac{\hat{\sigma}_{n}}{\sqrt{n}}g_{\frac{1+\alpha}{2}}, \hat{\theta}_{n}-\frac{\hat{\sigma}_{n}}{\sqrt{n}}g_{\frac{1-\alpha}{2}})$ (Efron's percentile interval)
   
Here you see that the percentile interval made errors. Hall (1992) called these problems 'the wrong table problem' ($q \neq g$) and the 'backward problem' (the wrong percentile, minus sign is replaced by the plus sign and the percentile is 'flipped'). So the conclusion we see is that the percentile method will work only when $G_{n}(.)=H_{n}(.)$ and the sampling distribution is symmetric.

Now let's create a percentile confidence interval as an example:

In [2]:
datavec = {'Price': [-11.9,22.3,34,6.27,14.56,-6.76,45.08,-9.12,56.34,67.43,0.95,9.4,57,-33.35,-78,96.45]}
df = pd.DataFrame(datavec, columns = ['Price'])

In [3]:
def Efron_boot_Percentile_CI(B, data, func, test_stat_name, random_state, alpha):
    """
    -This function feeds in a univariate DataFrame.
    -It computes the 100(1-alpha)% reversed bootstrap percentile interval for the population parameter.
    
    -Alpha Values must be 0.01, 0.05, or 0.1.
    -B is the bootstrap replicate.
    -The argument 'func' is the functional name (e.g. np.mean, np.median) or any user-defined functions
    -The argument 'alpha' can only take on 3 potential values.
    
    -The function returns a tuple of CI.
    
    """
    if alpha not in (0.01,0.05,0.1):
        print('Alpha Values must be 0.01, 0.05, or 0.1, try again!')
    else:
        if isinstance(data, pd.DataFrame):
            theta_hat_n=data.apply(func)[0]
            print(test_stat_name + ':' + str(theta_hat_n))
            print('bootstrap size: ', B, '\n')
            
            df_boot_list = []
            random.seed(random_state)
            for i in range(B):
                s = random.randint(1,B) # returning a random integer N s.t. a<= N<=b in random.randint(a,b)
                df_boot_pseudo=resample(data, random_state=s, replace=True).reset_index(drop=True) 
                df_boot_list.append(df_boot_pseudo)
            df_boot = pd.concat(df_boot_list, axis=1) # getting the bootstrap sample, with each column as one bootstrap sample
        
            deltas1=df_boot.apply(func) # computing the test statistic of each sample
            deltas2=df_boot.apply(func)-theta_hat_n # subtract everything by the original test statistic
            deltas2.sort_values(inplace=True)
            
            quantiles=deltas2.quantile([alpha/2, 1-(alpha/2)]).to_dict()
            print(int(100*(1-alpha)),'percent','CI for the population parameter:\n', "[", theta_hat_n-quantiles[1-(alpha/2)], theta_hat_n-quantiles[alpha/2], "]")
            print('CI quantile cutoffs: ', alpha/2, 1-(alpha/2))
            return (theta_hat_n-quantiles[1-(alpha/2)], theta_hat_n-quantiles[alpha/2])

In [4]:
Efron_boot_Percentile_CI(B=5000, data=df, func=np.mean, test_stat_name='sample mean', random_state=101, alpha=0.05)

sample mean:16.915625000000002
bootstrap size:  5000 

95 percent CI for the population parameter:
 [ -3.371093749999961 37.162734375 ]
CI quantile cutoffs:  0.025 0.975


(-3.371093749999961, 37.162734375)

### III. Bias-Corrected Bootstrap Percentile Interval (BC)

The Efron's original bootstrap confidence interval enjoys many nice properties, and it's simple to compute. Yet it's far from being pivotal. Thus its **coverage accuracy** is poor (coverage accuracy of a confidence region measures how well a confidence region maintains its coverage probability, that is, if we specify a 95% confidence region, does the confidence region really cover 95% all the time). Notice that the coverage accuracy is better for the percentile interval if the sampling distribution of the statistic is symmetric and has the same kurtosis as that of the normal distribution. But symmetry is not always attainable. Similarly, the reverse percentile interval uses that same wrong pivot in reverse. And it suffers from the same problem of 'backwardness'.

The bias-corrected confidence interval is based on the following assumption, which says there exists some bias term $b$ that need to be corrected so that we can make our confidence intervals better:

##### Assumption:
Suppose there exists a bias correction term $b$ as well as a montonically increasing transformation $\psi(.)$ such that:
   - A1: $Pr(\psi(\hat{\theta}_{n})-\psi(\theta)+b \le t)=\Psi(t)$ 
   - A2: $\Psi(t)$ is continuous, monotonically increasing, and symmetric. 
   - A3: Let $\Psi_{\alpha}$ denotes the $\alpha$th quantile of $\Psi(.)$, so that $\Psi(\Psi_{\alpha})=\Psi(\Psi^{-1}(\alpha))=\alpha$. 

The following lemma can be proved by tedious algebraic manipulation:

##### Lemma:
Assume A1,A2,A3. Then $Pr(\theta \le \psi^{-1}[\psi(\hat{\theta}_{n})+b+\Psi_{\alpha}])=\alpha$. 

Given the results above, the object $\psi(\hat{\theta}_{n})-\psi(\theta)+b$ can be used as a part of the confidence limit for $\theta$ if we can find the value of $b$. Now, recall that $J_{n}(t)=Pr(\hat{\theta}_{n} \le t|x \sim F)$ and $\hat{J}_{n}(t)=Pr^{*}(\hat{\theta}_{n}^{*} \le t|x^{*} \sim \bar{F}_{n})$. Writing down the bootstrap analog, $Pr(\psi(\hat{\theta}_{n})-\psi(\theta)+b \le t)=\Psi(t)$ becomes $Pr^{*}(\psi(\hat{\theta}_{n}^{*})-\psi(\hat{\theta}_{n})+b \le t)=\Psi(t)$. Thus, we can obtain an explicit form of the bias-correction term $b=\Psi^{-1}(\hat{J}_{n}(\hat{\theta}_{n}))$, due to the fact that $\hat{J}_{n}(\hat{\theta}_{n})=Pr^{*}(\hat{\theta}_{n}^{*} \le \hat{\theta}_{n}|x^{*} \sim \bar{F}_{n})=Pr^{*}(\psi(\hat{\theta}_{n}^{*})-\psi(\hat{\theta}_{n})+b \le b|x^{*} \sim \bar{F}_{n})=\Psi(b)$. By examining this form, we see that if $\Psi(.)$ is not symmetric, we should certainly add the bias-correction term into our consideration for confidence interval. If additionally, $\hat{\theta}_{n}$ is the sample median of $\hat{J}_{n}(.)$. then $\hat{J}_{n}(\hat{\theta}_{n})=0.5$ and $b=0$. We call this phenomenon **median-unbiasedness**. Moreover, one can see by definition that $\psi^{-1}(\psi(\hat{\theta}_{n})-b+\Psi_{\alpha})=\hat{J}_{n}^{-1}(\alpha)$, which means that $\psi^{-1}(\psi(\hat{\theta}_{n})-b+\Psi_{\alpha})$ can be treated as part of the percentile confidence limit. 

We have the following definition:

##### Definition:
Suppose $x_{1}, x_{2},...x_{n} \sim F(.)$ is a random sample and $\theta=T(F)$ is a functional parameter. Let $J_{n}(t)=Pr(\hat{\theta}_{n} \le t|x \sim F)$ and $\hat{J}_{n}(t)=Pr^{*}(\hat{\theta}_{n}^{*} \le t|x^{*} \sim \bar{F}_{n})$ where $\bar{F}_{n}$ is the EDF. Then a 100$\alpha$% bootstrap bias-corrected confidence interval for $\theta$ is given by $(\hat{J}_{n}^{-1}(\alpha_{L}), \hat{J}_{n}^{-1}(\alpha_{U}))$ where $\alpha_{L}=\Phi(2\hat{b}+z_{\frac{1-\alpha}{2}})$ and $\alpha_{U}=\Phi(2\hat{b}+z_{\frac{1+\alpha}{2}})$ with $\Phi(.)$ denoting the CDF of a standard normal distribution, and $b=\Phi^{-1}(\hat{J}_{n}(\hat{\theta}_{n}))$ and $z_{\alpha}=\Phi^{-1}(\alpha)$ is the standard normal critical value. 

In actual implementation, we usually take the convenience of setting $\Psi(.)=\Phi(.)$ and $\psi(.)=\phi(.)$ by invoking the CDF and pdf of the standard normal distribution. The bias term in practice is approximated by the proportion of the bootstrap estimates less than the original parameter estimate, that is, $\hat{b}=\Phi^{-1}(\hat{J}_{n}(\hat{\theta}_{n}))=\Phi^{-1}(\frac{1}{B}\sum_{b=1}^{B}I(\hat{\theta}_{n,b}^{*} \le \hat{\theta}_{n}))$. Hansen (2020) laid out the steps of the calculation for the $100(1-\alpha)$% C.I.:
   1. Given the bootstrapped data, calculate $\hat{b}=\Phi^{-1}(\frac{1}{B}\sum_{b=1}^{B}I(\hat{\theta}_{n,b}^{*} \le \hat{\theta}_{n}))$.
   2. Define $z_{\alpha}=\Phi^{-1}(\alpha)$ and the function $f(\alpha)=\Phi(2\hat{b}+z_{\alpha})$.
   3. Calculate $f(\frac{\alpha}{2})$ and $f(1-\frac{\alpha}{2})$. 
   4. Obtain the (empirical) confidence interval $(q_{f(\frac{\alpha}{2})}^{*}, q_{f(1-\frac{\alpha}{2})}^{*})$.

In [5]:
def BC_CI(B, data, test_stat_func, test_stat_name, random_state, alpha):
    """
    
    """
    if alpha not in (0.01,0.05,0.1):
        print('Alpha Values must be 0.01, 0.05, or 0.1, try again!')
    else:
        if isinstance(data, pd.DataFrame):
            theta_hat_n=data.apply(test_stat_func)[0]
            print(test_stat_name + ':' + str(theta_hat_n))
            print('bootstrap size: ', B, '\n')
            
            df_boot_list = []
            random.seed(random_state)
            for i in range(B):
                s = random.randint(1,B) # returning a random integer N s.t. a<= N<=b in random.randint(a,b)
                df_boot_pseudo=resample(data, random_state=s, replace=True).reset_index(drop=True) 
                df_boot_list.append(df_boot_pseudo)
            df_boot = pd.concat(df_boot_list, axis=1) # getting the bootstrap sample, with each column as one bootstrap sample
        
            theta_hat_nb=df_boot.apply(test_stat_func) # computing the test statistic of each bootstrap sample
            indicator_list=[num for num in theta_hat_nb if num <= theta_hat_n]
            b_hat=norm.ppf(len(indicator_list)/B)
            # print('b_hat (bias-correction term):', b_hat)
            
            def f(x):
                y=norm.cdf(2*b_hat+norm.ppf(x))
                return y
            alpha_low=f(alpha/2)
            alpha_high=f(1-alpha/2)
            
            theta_hat_nb.sort_values(inplace=True)
            quantiles=theta_hat_nb.quantile([alpha_low, alpha_high]).to_list()
            print(int(100*(1-alpha)),'percent','CI for the population mean:\n', "[", quantiles[0], quantiles[1], "]")
            print('CI quantile cutoffs: ', alpha/2, 1-(alpha/2))
            return (quantiles[0], quantiles[1])

In [6]:
BC_CI(B=10000, data=df, test_stat_func=np.mean, test_stat_name='sample mean', random_state=102, alpha=0.05)

sample mean:16.915625000000002
bootstrap size:  10000 

95 percent CI for the population mean:
 [ -4.206126943670495 37.090522354325095 ]
CI quantile cutoffs:  0.025 0.975


(-4.206126943670495, 37.090522354325095)

The bias-correted bootstrap confidence interval was later revised by Efron (1987) who introduced one additional parameter called the 'acceleration parameter'. Efron called the new confidence interval the bias-corrected and accelerated percentile method. The accleration parameter $a$ is normall assumed to be positive. If $a=0$, then we will be going back to the BC interval. Efron (1993) has a good introduction to this topic and the calculatio of $a$ involves jackknife. We will not elaborate the details here. 

### III. Studentized Bootstrap Confidence Interval

In many cases we can obtain improvement in accuracy by bootstrapping a studentized type of statistic such as a t-ratio. The motivation is that all percentile type of bootstrap confidence intervals are subject to certain finite sample bounds on their coverage probability. After all, all the intervals so far are not utilizing any of the standard error. Recall that the sample t-ratio is given by $T=\frac{\hat{\theta}_{n}-\theta}{se(\hat{\theta}_{n})}$. The bootstrap analog becomes $T_{b}^{*}=\frac{\hat{\theta}_{n,b}^{*}-\hat{\theta}_{n}}{se(\hat{\theta}_{n,b}^{*})}$ where $se(\hat{\theta}_{n,b}^{*})$ denotes the standard error calculated on the $b$th bootstrap sample. We can then do this B times and store all values of $T_{b}^{*}$. The $\alpha$th quantile $q_{\alpha}$ is estimated by the empirical quantile $q^{*}_{\alpha}$ from the $B$ bootstrap draws of $T_{b}^{*}$. The $100(1-\alpha)$% bootstrap percentile-T confidence interval is then defined as $(\hat{\theta}_{n}-se(\hat{\theta}_{n})q^{*}_{1-\frac{\alpha}{2}}, \hat{\theta}_{n}-se(\hat{\theta}_{n})q^{*}_{\frac{\alpha}{2}})$. Details are highlighted in Hansen (2020). 

There are other different ways of implementing this confidence interval (and slightly different versions of the studentized interval. The general consensus is that these studentized confidence interval has a much smaller coverage probability than any Efron's percentile intervals. But the downside is that the implementation requires large sample size because sometimes your standard error could be zero when you repeatedly draw data from the original sample. 

Below we create a simplifed version of the studentized version of the interval. Notice that below we define sample standard deviation to be $s=\sqrt{\frac{\sum_{i=1}^{n}(x_{i}-\overline{x})^{2}}{n-1}}$ and $\hat{se}=\frac{s}{\sqrt{n}}$. Here we are assuming that the statistic is nice and $se(\hat{\theta}_{n})$ is easy to obtain (simply using the standard error of the mean). 

Recall that the standard error of a statistic (usually an estimate of a parameter) is the standard deviation of its sampling distribution. If the statistic is the sample mean, it is called the standard error of the mean (SEM). In many cases, the actual standard error of the estimator in the sample is not easy to obtain. So people have suggested doing bootstrap within bootstrap to get the standard error as an estimate of. We will ignore the details here and present a simplified version of the studentized interval below using stadard error of the mean as a default option:

In [7]:
def studentized_bootstrap_CI_v0(B, data, func, test_stat_name, random_state, alpha):
    """
    -This function feeds in a univariate DataFrame.
    -It computes the 100(1-alpha)% studentized bootstrap confidencen interval for the population parameter.
    
    -Alpha Values must be 0.01, 0.05, or 0.1.
    -B is the bootstrap replicate.
    -The argument 'func' is the functional name (e.g. np.mean, np.median) or any user-defined functions
    -The argument 'alpha' can only take on 3 potential values.
    
    -The function returns a tuple of CI.
    
    """
    if alpha not in (0.01,0.05,0.1):
        print('Alpha Values must be 0.01, 0.05, or 0.1, try again!')
    else:
        if isinstance(data, pd.DataFrame):
            theta_hat_n=data.apply(func)[0]
            se_hat=data.apply(stats.sem)[0]
            print(test_stat_name + ':' + str(theta_hat_n))
            print('bootstrap size: ', B, '\n')
            
            df_boot_list = []
            random.seed(random_state)
            for i in range(B):
                s = random.randint(1,B) # returning a random integer N s.t. a<= N<=b in random.randint(a,b)
                df_boot_pseudo=resample(data, random_state=s, replace=True).reset_index(drop=True) 
                df_boot_list.append(df_boot_pseudo)
            df_boot = pd.concat(df_boot_list, axis=1) # getting the bootstrap sample, with each column as one bootstrap sample
        
            theta_hat_nb_star=df_boot.apply(func) # computing the test statistic of each sample
            se_hat_star=df_boot.apply(stats.sem)
            T=(theta_hat_nb_star-theta_hat_n)/se_hat_star
            T.sort_values(inplace=True)
            
            quantiles=T.quantile([alpha/2, 1-(alpha/2)]).to_dict()
            print(int(100*(1-alpha)),'percent','CI for the population parameter:\n', "[", theta_hat_n-se_hat*quantiles[1-(alpha/2)], theta_hat_n-se_hat*quantiles[alpha/2], "]")
            print('CI quantile cutoffs: ', alpha/2, 1-(alpha/2))
            return (theta_hat_n-se_hat*quantiles[1-(alpha/2)], theta_hat_n-se_hat*quantiles[alpha/2])

In [8]:
studentized_bootstrap_CI_v0(B=10000, data=df, func=np.mean, test_stat_name='sample mean', random_state=102, alpha=0.05)

sample mean:16.915625000000002
bootstrap size:  10000 

95 percent CI for the population parameter:
 [ -6.776319382670927 39.193399157373484 ]
CI quantile cutoffs:  0.025 0.975


(-6.776319382670927, 39.193399157373484)

### III. Hypothesis Testing

The idea of hypothesis testing in the bootstrap framework can be applied using the following lemma (see Casella and Berger PP 397-398):

##### Lemma:
Suppose $x_{1}, x_{2},...x_{n} \sim F(.)$ is a random sample and $\theta=T(F)$ is a functional parameter. Let $w(x)$ be a test statistic such that large values of $w(x)$ gives evidence that the alternative hypothesis $H_{1}$ is true. For any x, define $p(x)=sup_{\theta \in \Theta_{0}} Pr(w(x) \ge w|\theta)$. Then p(x) is a valid p-value, with $\Theta_{0} \subseteq \Theta$. 

Most of the time, we assume that null parameter space  $\Theta_{0}=\{\theta_{0}\}$ (simple hypothesis testing). And using p-value to do hypothesis testing is an easy way to go about solving the problem. If the distribution of $w(x)$ under the null hypothesis is unknow, then we can estimate the p-value using bootstrap. So in the bootstrap world, we can set $\hat{p}=Pr^{*}(w^{*} \ge w_{0}|x^{*}_{1},...x^{*}_{n} \sim \hat{F}_{0}, H_{0})$. In general,  $\hat{p}=sup_{\theta \in \Theta_{0}}Pr^{*}(w^{*} \ge w_{0}|x^{*}_{1},...x^{*}_{n} \sim \hat{F}_{0}, H_{0})$. But we will drop the 'sup' operator because we are dealing with only simple hypotheses. So really the question boils down to the choice of $\hat{F}_{0}$ as well as the test statistic. We can do some analysis using the following scenarios:

##### Case I: when $\theta$ is a location parameter
If this is the case, we have $F_{\theta}=F(x-\theta)$. Under $H_{0}: \theta=\theta_{0}$, we set $w=\hat{\theta}_{n}-\theta_{0}$. Let $\theta$ denote the population mean, then let $y_{i}=x_{i}-\hat{\theta}_{n}+\theta_{0}$. Clearly the new sample $y_{i}$ has a sample mean of $\theta_{0}$ under the null hypothesis. So $\hat{F}_{0}=\frac{1}{n}\sum_{i=1}^{n}I_{(-\infty,t)}(y_{i})$. 

##### Case II: when $\theta$ is a scale parameter
If this is the case, we have $F_{\theta}=F(\frac{x}{\theta})$. Under $H_{0}: \theta=\theta_{0}$, we set $w=\frac{\hat{\theta}_{n}}{\theta_{0}}$. Let $\theta$ denote the population variance, then let $y_{i}=\sqrt{\frac{\theta_{0}}{\hat{\theta}_{n}}}x_{i}$. Clearly the new sample $y_{i}$ has a sample variance of 1 under the null hypothesis. So $\hat{F}_{0}=\frac{1}{n}\sum_{i=1}^{n}I_{(-\infty,t)}(y_{i})$. 

The standard econometrics texts by Hansen(2020) as well as Cameron & Pravin K. (2005) have a furhter treatment on this topic. We will not elaborate all the details here. 

#### References:
   - Bhattacharya, R. N.; Ghosh, J. K. On the Validity of the Formal Edgeworth Expansion. Ann. Statist. 6 (1978), no. 2, 434--451. 
   - Casella, G., & Berger, R. L. (2002). Statistical inference. Belmont, CA: Duxbury.
   - Davison and Hinkley (1997) Bootstrapping and its Applications, Cambridge University Press.
   - Efron, B. (1982) The Jackknife, the Bootstrap and Other Resampling Plans, SIAM.
   - Efron, B. (1987). Better bootstrap confidence intervals. J. Am. Stat. Assoc. 82, 171–185.
   - Efron, B. and Tibshirani, R.J. (1993) An Introduction to the Bootstrap, Chapman & Hall.
   - Manly, Bryan F.J. (2007) Randomization, Bootstrap, and Monte Carlo Methods in Biology, Chapman & Hall/CRC Press.
   - Hansen, B. (2020) Econometrics, University of Wisconsin-Madison, Manuscript. 
   - Shao, J. and Tu, T. (1995) The Jackknife and Bootstrap, Springer-Verlag.
   - Cameron, A. Colin and Trivedi, Pravin K. (2005) Microeconometrics: Methods and Applications. Cambridge University Press.
   - Singh, K. (1981), "On the Asymptotic Accuracy of Efron’s Bootstrap", Annals of Statistics, 9,
1187–1195.
   - Hall, P. (1992), The Bootstrap and Edgeworth Expansions, New York, Springer-Verlag.
   - Polansky, A. (2014), STAT673: Lecture Notes on Bootstrap, NIU. 
   - https://www.frontiersin.org/articles/10.3389/fpsyg.2019.02215/full
   - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4784504/#:~:text=Reverse%20Bootstrap%20Percentile%20Interval,like%20those%20shown%20just%20below.